In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\SummarEase\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\SummarEase'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
%pwd

'e:\\Projects\\SummarEase Project\\research'

In [1]:
import sys
sys.path.append('E:\\Projects\\SummarEase-Project\\src')


In [2]:
import SummarEase

In [3]:
from SummarEase.constants import *
from SummarEase.utils.common import read_yaml, create_directories

In [4]:
%pwd

'e:\\Projects\\SummarEase-Project\\research'

In [15]:
from pathlib import Path

class ConfigurationManager:
    def __init__(self, config_filepath=None, params_filepath=None):
        self.config_filepath = Path(config_filepath or CONFIG_FILE_PATH)
        self.params_filepath = Path(params_filepath or PARAMS_FILE_PATH)
        
        self.config = read_yaml(self.config_filepath)
        self.params = read_yaml(self.params_filepath)
        
        create_directories([self.config.artifacts_root])
           
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from SummarEase.logging import logger
from SummarEase.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
CONFIG_FILE_PATH = 'E:\\Projects\\SummarEase-Project\\config\\config.yaml'
PARAMS_FILE_PATH = 'E:\\Projects\\SummarEase-Project\\params.yaml'

try:
    config = ConfigurationManager(config_filepath=CONFIG_FILE_PATH)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-25 16:48:44,532: INFO: common: yaml file: E:\Projects\SummarEase-Project\config\config.yaml loaded successfully]
[2024-04-25 16:48:44,619: INFO: common: yaml file: E:\Projects\SummarEase-Project\params.yaml loaded successfully]
[2024-04-25 16:48:44,621: INFO: common: created directory at: artifacts]
[2024-04-25 16:48:44,624: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-25 16:49:14,190: INFO: 2712961683: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8B96:3516D2:8A7DA6:A4518C:662A3C2B
Accept-Ranges: bytes
Date: Thu, 25 Apr 2024 11:19:08 G